In [1]:
import numpy as np
import tensorflow as tf


print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

2.3.2


In [2]:
dataset = np.load("dataset_overlap.npy")
classes = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/bin_classes.npy")

In [3]:
def unison_shuffled_copies(a, b):
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [75]:
def split(ds, labels: np.ndarray, perc):
    split_point = int(np.ceil(len(ds)*perc))
    val_dim = len(ds)-split_point
    ones = 0
    zeros = 0
    train_inserted = 0
    val_inserted = 0
    train_ds = np.zeros((split_point,32,16516))
    train_labels = np.zeros((split_point,), dtype=np.int64)
    val_ds = np.zeros((val_dim, 32, 16516))
    val_labels = np.zeros((val_dim,),  dtype=np.int64)
    for data, label in zip(ds,labels):
        if label == 0:
            if zeros < np.ceil(split_point/2):
                train_ds[train_inserted] = data
                train_labels[train_inserted] = label
                zeros +=1
                train_inserted += 1
            else:
                val_ds[val_inserted] = data
                val_labels[val_inserted] = label
                val_inserted += 1
        if label == 1:
            if ones < np.ceil(split_point/2):
                train_ds[train_inserted] = data
                train_labels[train_inserted] = label
                ones +=1
                train_inserted += 1
            else:
                val_ds[val_inserted] = data
                val_labels[val_inserted] = label
                val_inserted += 1

    return train_ds, train_labels, val_ds, val_labels

In [4]:
def normalize(array, std_dev, mean, scale):
    array = array/scale
    return array-mean/std_dev

In [77]:
def split_dataset_and_normalize(perc=0.8):
    shuffled_dataset, shuffled_labels = unison_shuffled_copies(dataset,classes)
    split_point = int(len(dataset)*perc)
    train_ds, train_labels, val_ds, val_labels = split(shuffled_dataset, shuffled_labels, perc)
    std_dev = np.std(train_ds)
    mean = np.mean(train_ds)
    maximum = np.max(train_ds)
    min = np.min(train_ds)
    scale = max(maximum, abs(min))
    train_ds = normalize(train_ds, std_dev, mean, scale)
    val_ds = normalize(val_ds, std_dev, mean, scale)
    return train_ds, train_labels, val_ds, val_labels

train_ds, train_labels, val_ds, val_labels = split_dataset_and_normalize()

In [78]:
print((train_labels == 0).sum())
print((train_labels == 1).sum())
print((val_labels == 0).sum())
print((val_labels == 1).sum())

297
297
45
103


BIDIRECTIONAL LSTM

In [ ]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer, GRU, LSTMCell, RNN, Bidirectional
from tensorflow import keras
from tensorflow.keras import  regularizers
input_shape=(32,16516)
model = keras.Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(Bidirectional(LSTM(192, input_shape=input_shape,
                             kernel_regularizer=regularizers.l2(0.00001),
                                 recurrent_regularizer=regularizers.l2(0.00001),
                                 bias_regularizer=regularizers.l2(0.00001),
                             recurrent_dropout=0.3)))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

1-DIR LSTM #1

In [8]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer, GRU, LSTMCell, RNN, Bidirectional
from tensorflow import keras
from tensorflow.keras import  regularizers
input_shape=(32,16516)
model = keras.Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(LSTM(128, input_shape=input_shape,
                             kernel_regularizer=regularizers.l2(0.00001),
                                 recurrent_regularizer=regularizers.l2(0.00001),
                                 bias_regularizer=regularizers.l2(0.00001),
                             recurrent_dropout=0.4))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               8522240   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

1-DIR LSTM #2

In [41]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer, GRU, LSTMCell, RNN, Bidirectional
from tensorflow import keras
from tensorflow.keras import  regularizers
input_shape=(32,16516)
model = keras.Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(LSTM(96, input_shape=input_shape,
                             kernel_regularizer=regularizers.l2(0.000001),
                                 recurrent_regularizer=regularizers.l2(0.000001),
                                 bias_regularizer=regularizers.l2(0.000001),
                             recurrent_dropout=0.4))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 96)                6379392   
_________________________________________________________________
dropout_25 (Dropout)         (None, 96)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 64)                6208      
_________________________________________________________________
dropout_26 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_27 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                

1-DIR LSTM #3

In [90]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer, GRU, LSTMCell, RNN, Bidirectional
from tensorflow import keras
from tensorflow.keras import  regularizers
input_shape=(32,16516)
model = keras.Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(LSTM(64, input_shape=input_shape,
                             kernel_regularizer=regularizers.l2(0.00005),
                                 recurrent_regularizer=regularizers.l2(0.00005),
                                 bias_regularizer=regularizers.l2(0.00005),
                             recurrent_dropout=0.4))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 64)                4244736   
_________________________________________________________________
dropout_40 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_40 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_41 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_41 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_42 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)               

In [81]:
filepath = "audio_model_LSTM_overlap_3.h5"

In [91]:
cb1 = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)
cb2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)

cbs = [cb1,cb2]

In [92]:
#callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00003),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy'],
)

In [93]:
EPOCHS = 100
history = model.fit(
    x= train_ds,
    y = train_labels,
    validation_data=(val_ds, val_labels),
    batch_size=16,
    epochs=EPOCHS,
    callbacks=cbs
)

Epoch 1/100
38/38 [==============================] - 3s 86ms/step - loss: 0.7242 - accuracy: 0.4764 - val_loss: 0.7239 - val_accuracy: 0.4122

In [89]:
del model

In [52]:
np.save('history_no_bid3.npy',history.history)

In [12]:
model2 = keras.models.load_model("audio_model_bidirectional_LSTM_overlap.h5")

model2.evaluate(val_ds)

In [74]:
del train_ds, train_labels, val_ds, val_labels